# Config

Folder Structure on EE

- mars
    - features
        - {feature_name}
            - pre
            - post
    - labels
        - {label_name}
            - pre
            - post

In [ ]:
import ee
import folium
import geopandas as gpd
import tensorflow as tf

In [67]:
ee.Initialize()

In [68]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee_image_object.getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name = name,
        overlay = True,
        control = True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [ ]:
mars_hrsc_sample = ee.ImageCollection("projects/esg-satelite/assets/mars/mars_express_hrsc_example")
faultline_vectors = ee.FeatureCollection("projects/esg-satelite/assets/mars/labels/faultlines")
# faultlines_raster = ee.Image("projects/esg-satelite/assets/mars/labels/faultlines_raster")

In [ ]:
BANDS = ["HRSC"]
RESPONSE = "FAULTLINE"
FEATURES = BANDS + [RESPONSE]

KERNEL_SIZE = 64
COLUMNS = [
    tf.io.FixedLenFeature(shape=[KERNEL_SIZE, KERNEL_SIZE], dtype=tf.float32) for _ in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

In [ ]:
GEOMETRY_WORLD = ee.Geometry.Polygon(
    [
        [-180, 88],
        [180, 88],
        [180, -88],
        [-180, -88],
    ],
    None,
    False
)

# HRSC Data Processing

#### Black band removal

In [205]:
mars_hrsc_sample = mars_hrsc_sample.select(["b1"], ["HRSC"])

In [107]:
def pixel_mask(image):
    return image.updateMask((image.select("HRSC").gt(0)))

mars_hrsc_filtered = mars_hrsc_sample.map(pixel_mask)
mars_hrsc_mosaic = mars_hrsc_filtered.mosaic()

Just to check if the black pixels are actually 0.

In [92]:
region = mars_hrsc_sample.geometry()
maxReducer = ee.Reducer.min()
print(mars_hrsc_mosaic.reduceRegion(maxReducer, region, 300, bestEffort=True).getInfo())

{'b1': 0}


In [ ]:
task = ee.batch.Export.image.toAsset(
    mars_hrsc_mosaic,
    description = "HRSC_Image",
    assetId="projects/esg-satelite/assets/mars/features/hrsc_sample/post",
    region=GEOMETRY_WORLD,
    scale=1000,
    maxPixels=1e11
    #Note that crs is a parameter here - look this up in future
)

In [ ]:
task.start()

# Fault Line Data Processing

Add a column of ones for reduction.

In [206]:
def add_column(feature):
    return feature.set({"Value": 1})

faultline_vectors_with_ones = faultline_vectors.map(add_column)

In [123]:
faultline_im = (
    faultline_vectors_with_ones.reduceToImage(
        properties=["Value"],
        reducer=ee.Reducer.first()
    )
    .select(["first"], ["FAULTLINE"])
    .unmask(0)
)

In [129]:
task = ee.batch.Export.image.toAsset(
    faultline_im,
    description = "Faultline_image",
    assetId="projects/esg-satelite/assets/mars/labels/faultlines_raster",
    region=GEOMETRY_WORLD,
    scale=1000,
    maxPixels=1e11
    #Note that crs is a parameter here - look this up in future
)

In [130]:
task.start()

Stacking the mars image onto the faultlines raster

In [165]:
faultlines_raster = ee.Image("projects/esg-satelite/assets/mars/labels/faultlines_raster")
#faultlines_raster = faultline_im
# REMEMBER TO UNMASK AND RESAVE THE IMAGE!

In [132]:
# Make sure to save the mars mosaiced image so you can easily read it back in
mars = mars_hrsc_filtered.mosaic()

In [134]:
image_stack = ee.Image.cat(
    [
        faultlines_raster,
        mars
    ]
)

In [135]:
image_stack = image_stack.float()

In [144]:
# Makes a 64x64 tensor of 1s
ee_list = ee.List.repeat(1,KERNEL_SIZE)
ee_lists = ee.List.repeat(ee_list,KERNEL_SIZE)
kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, ee_lists)

In [145]:
# Adds image array of 64x64 around each pixel
image_stack_neighbours = image_stack.neighborhoodToArray(kernel)

In [198]:
# Makes a feature collection of "patches" which are regions (geometries) of different sets of data
PATCHES_JSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type":"Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              -75.69403257726853,
              45.43314727092101
            ],
            [
              -75.69403257726853,
              41.20493154723766
            ],
            [
              -69.91084889191063,
              41.20493154723766
            ],
            [
              -69.91084889191063,
              45.43314727092101
            ],
            [
              -75.69403257726853,
              45.43314727092101
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

PATCHES = ee.FeatureCollection(PATCHES_JSON)

In [199]:
patch_list = PATCHES.toList(PATCHES.size())
task_list = []

# Extracts feature from the feature collection, and gets the geom property
for geometry_index in range(PATCHES.size().getInfo()):
    image_stack_sample = image_stack_neighbours.sample(
        region=ee.Feature(patch_list.get(geometry_index)).geometry(),
        scale=1000,
        numPixels=1000,
        seed=123
    )

    desc = f"Flood Data Geometry: {geometry_index}"
    task = ee.batch.Export.table.toCloudStorage(
        collection=image_stack_sample,
        description=f"Flood Data Region {geometry_index}",
        bucket="esg-satelite-data-warehouse",
        fileNamePrefix=f"natural-disaster-monitoring/modelling/train/training_data_{geometry_index}",
        fileFormat = "TFRecord"
    )
    task_list.append(task)

In [200]:
task_list

[<Task EXPORT_FEATURES: Flood Data Region 0 (UNSUBMITTED)>]

In [202]:
image_stack_sample.size().getInfo()

238